In [10]:
from pytorch3d.transforms import euler_angles_to_matrix, matrix_to_quaternion, matrix_to_quaternion, matrix_to_euler_angles
import torch

/root/miniconda3/envs/smplerx/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
tests = [torch.tensor([1.0, 190.0, 35.0]), 
                torch.tensor([263.0, 190.0, -85.0]), 
                torch.tensor([274.0, -1.0, 8.0]), 
                torch.tensor([torch.pi / 2, 0.0, 0.0]),
                torch.tensor([0.0, torch.pi / 2, 0.0]),
                torch.tensor([0.0, 0.0, torch.pi / 2]),]


# for euler in tests:
#     # euler = torch.tensor([euler[2], euler[1], euler[0]])
#     for rot_format in ["ZXY"]:
#         R = euler_angles_to_matrix(euler, rot_format)
#         q = matrix_to_quaternion(R)
#         print(rot_format, "   x: {:.4f}, y: {:.4f}, z: {:.4f}, w: {:.4f}".format( q[1], q[2], q[3], q[0]))

ZXY    x: -0.4731, y: 0.5082, z: 0.5533, w: -0.4602
ZXY    x: 0.3688, y: 0.5859, z: 0.6298, w: 0.3523
ZXY    x: -0.5213, y: 0.6619, z: -0.5173, w: 0.1503
ZXY    x: 0.0000, y: 0.7071, z: 0.0000, w: 0.7071
ZXY    x: 0.7071, y: 0.0000, z: 0.0000, w: 0.7071
ZXY    x: 0.0000, y: 0.0000, z: 0.7071, w: 0.7071


In [12]:
class sincos:
    def __init__(self, angle: torch.Tensor) -> None:
        self.s = torch.sin(angle)
        self.c = torch.cos(angle)

def unity_euler2quaternion(euler):
    euler = euler * 0.5
    xa, ya, za = sincos(euler[..., 0]), sincos(euler[..., 1]), sincos(euler[..., 2])
    
    x = xa.s * ya.c * za.c + ya.s * za.s * xa.c
    y = ya.s * xa.c * za.c - xa.s * za.s * ya.c
    z = za.s * xa.c * ya.c - xa.s * ya.s * za.c
    w = xa.c * ya.c * za.c + ya.s * za.s * xa.s
    return torch.stack([x, y, z, w], dim=-1)

for test in tests:
    q = unity_euler2quaternion(test)
    print("unityeuler2quaternion", "   x: {:.4f}, y: {:.4f}, z: {:.4f}, w: {:.4f}".format(q[0], q[1], q[2], q[3]))
    print("norm: ", torch.norm(q))

unityeuler2quaternion    x: -0.5082, y: 0.4731, z: -0.6971, w: -0.1790
norm:  tensor(1.0000)
unityeuler2quaternion    x: 0.5859, y: 0.3688, z: 0.6820, w: -0.2359
norm:  tensor(1.0000)
unityeuler2quaternion    x: 0.6619, y: -0.5213, z: 0.0734, w: -0.5337
norm:  tensor(1.)
unityeuler2quaternion    x: 0.7071, y: 0.0000, z: 0.0000, w: 0.7071
norm:  tensor(1.0000)
unityeuler2quaternion    x: 0.0000, y: 0.7071, z: 0.0000, w: 0.7071
norm:  tensor(1.0000)
unityeuler2quaternion    x: 0.0000, y: 0.0000, z: 0.7071, w: 0.7071
norm:  tensor(1.0000)


In [33]:
from sympy import Quaternion, pi, cos, sin, Matrix, solve
from sympy import symbols, I
import numpy as np

rot_x = symbols('x')
rot_y = symbols('y')
rot_z = symbols('z')

q_rot_x = Quaternion(cos(rot_x / 2), sin(rot_x / 2), 0, 0)
q_rot_y = Quaternion(cos(rot_y / 2), 0, sin(rot_y / 2), 0)
q_rot_z = Quaternion(cos(rot_z / 2), 0, 0, sin(rot_z / 2))

In [34]:
q = q_rot_z * q_rot_x * q_rot_y
equations = [q - Quaternion(symbols('qw'), symbols('qx'), symbols('qy'), symbols('qz'))]

solutions = solve(equations, [rot_x, rot_y, rot_z], dict=True)

In [37]:
solutions

[]

In [39]:
alpha, beta, gamma = symbols('alpha beta gamma')

rot_x = Matrix([[1, 0, 0], [0, cos(alpha), -sin(alpha)], [0, sin(alpha), cos(alpha)]])
rot_y = Matrix([[cos(beta), 0, sin(beta)], [0, 1, 0], [-sin(beta), 0, cos(beta)]])
rot_z = Matrix([[cos(gamma), -sin(gamma), 0], [sin(gamma), cos(gamma), 0], [0, 0, 1]])


In [45]:
rot_y * rot_z * rot_x # euclideanspace

Matrix([
[ cos(beta)*cos(gamma), sin(alpha)*sin(beta) - sin(gamma)*cos(alpha)*cos(beta),  sin(alpha)*sin(gamma)*cos(beta) + sin(beta)*cos(alpha)],
[           sin(gamma),                                  cos(alpha)*cos(gamma),                                  -sin(alpha)*cos(gamma)],
[-sin(beta)*cos(gamma), sin(alpha)*cos(beta) + sin(beta)*sin(gamma)*cos(alpha), -sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(beta)]])

In [46]:
rot_z * rot_x * rot_y # R_unity

Matrix([
[-sin(alpha)*sin(beta)*sin(gamma) + cos(beta)*cos(gamma), -sin(gamma)*cos(alpha),  sin(alpha)*sin(gamma)*cos(beta) + sin(beta)*cos(gamma)],
[ sin(alpha)*sin(beta)*cos(gamma) + sin(gamma)*cos(beta),  cos(alpha)*cos(gamma), -sin(alpha)*cos(beta)*cos(gamma) + sin(beta)*sin(gamma)],
[                                  -sin(beta)*cos(alpha),             sin(alpha),                                    cos(alpha)*cos(beta)]])